In [1]:
import findspark
findspark.init("/home/gorazda/spark-2.4.7-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("regression_evaluation").getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
for item in data.head(2)[1]:
    print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [13]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 
                                       'Time on App',
                                       'Time on Website',
                                       'Length of Membership'],
                            outputCol="features")

In [14]:
output = assembler.transform(data)

In [15]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [18]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [19]:
final_data = output.select("features", "Yearly Amount Spent")

In [20]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [21]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [23]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                353|
|   mean| 497.63822900635716|
| stddev|  80.83485715285724|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [24]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                147|
|   mean|  503.3382604765397|
| stddev|  75.65648950738326|
|    min|  304.1355915788555|
|    max|  765.5184619388373|
+-------+-------------------+



In [26]:
lr = LinearRegression(labelCol="Yearly Amount Spent")

In [27]:
lr_model = lr.fit(train_data)

In [28]:
test_results = lr_model.evaluate(test_data)

In [29]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|0.44333557931071255|
| 11.125535090408334|
|-3.6987814323949237|
|  4.329356543397921|
|-20.925270584578698|
| -7.550162049806431|
| 10.394635262422753|
|  3.215149334426826|
|  18.31877649629996|
| 17.541808044217817|
|-0.5259034026379368|
|  -26.3692950735782|
| -7.516363506205039|
|-1.3462132456158429|
| -4.978364000460999|
|-1.5457259382885127|
| -9.323857548109743|
|   8.73634508347942|
|-10.260118868857376|
|-17.091319383058647|
+-------------------+
only showing top 20 rows



In [30]:
test_results.rootMeanSquaredError

10.32134833382117

In [31]:
test_results.r2

0.9812610618616194

In [32]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [33]:
unlabeled_data = test_data.select("features")

In [34]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.5743636841713...|
|[30.7377203726281...|
|[30.8364326747734...|
|[31.0472221394875...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.5316044825729...|
|[31.6005122003032...|
|[31.6098395733896...|
|[31.6610498227460...|
|[31.6739155032749...|
|[31.7207699002873...|
|[31.7216523605090...|
|[31.7656188210424...|
|[31.8186165667690...|
|[31.8279790554652...|
|[31.8512531286083...|
|[31.8648325480987...|
|[31.9048571310136...|
+--------------------+
only showing top 20 rows



In [35]:
predictions = lr_model.transform(unlabeled_data)

In [36]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...|441.62107817875494|
|[30.7377203726281...|450.65520710582155|
|[30.8364326747734...|471.20068185938453|
|[31.0472221394875...| 388.1680426456235|
|[31.1239743499119...|507.87232442434447|
|[31.1280900496166...| 564.8028487968611|
|[31.1695067987115...|416.96189553987006|
|[31.5316044825729...|433.30045639493574|
|[31.6005122003032...|  460.854074994797|
|[31.6098395733896...|427.00374160689034|
|[31.6610498227460...| 416.8842569825388|
|[31.6739155032749...| 502.0943629834594|
|[31.7207699002873...|  546.291296984228|
|[31.7216523605090...| 349.1231398774885|
|[31.7656188210424...|501.53244563606813|
|[31.8186165667690...|447.96439930842416|
|[31.8279790554652...|449.32660509505126|
|[31.8512531286083...|  464.255901583319|
|[31.8648325480987...|450.15139934567105|
|[31.9048571310136...| 491.0411768058748|
+--------------------+------------